In [5]:
'''
Name: James Davis
Date: May 18, 2022
Course: Ban601
Assignment: Module_2

Task:
1)	What are the specific anomalies you uncovered in your dataset? 
    Note: Be sure they are different from the ones Thomas pointed out. 
    Why are they anomalies? 
    How did you specifically address them? NOTE: be specific and precise.
2)	From your First Cut Predictive Model, what coefficient estimates have 
    signs that do not seem to make sense from a theoretical or business perspective? 
    Explain why those signs do not make sense.
3)	What are both (1) the direction (under-predicting or over-predicting)
    and (2) magnitude of the forecast errors from this First Cut Predictive Model?

Given:
West_Roxbury_Data_Set_Only.xlsx

'''

'\nName: James Davis\nDate: May 18, 2022\nCourse: Ban601\nAssignment: Module_2\n\nTask:\n1)\tWhat are the specific anomalies you uncovered in your dataset? \n    Note: Be sure they are different from the ones Thomas pointed out. \n    Why are they anomalies? \n    How did you specifically address them? NOTE: be specific and precise.\n2)\tFrom your First Cut Predictive Model, what coefficient estimates have \n    signs that do not seem to make sense from a theoretical or business perspective? \n    Explain why those signs do not make sense.\n3)\tWhat are both (1) the direction (under-predicting or over-predicting)\n    and (2) magnitude of the forecast errors from this First Cut Predictive Model?\n\nGiven:\nWest_Roxbury_Data_Set_Only.xlsx\n\n'

In [6]:
#Import Libs
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [7]:
#Load Dataset as train 
#Note: Load sheet 2 'Data'
df_train = pd.read_excel('WEST ROXBURY DATA SET ONLY.xlsx','Data')

FileNotFoundError: [Errno 2] No such file or directory: 'WEST ROXBURY DATA SET ONLY.xlsx'

In [ ]:
#Show Vars
print(df_train.columns)

In [ ]:
#Clean up name values 
#Noticed a space after some headers
df_train.rename(columns = {'TOTAL VALUE ': 'Total_Value'
                           ,'TAX': 'Tax_Cost'
                           ,'LOT SQFT ': 'Lot_SQFT'
                           ,'YR BUILT': 'Year_Built'
                           ,'GROSS AREA ': 'Gross_Area'
                           ,'LIVING AREA': 'Living_Area'
                           ,'FLOORS': 'Floor_Count'
                           ,'ROOMS': 'Room_Count'
                           ,'BEDROOMS ': 'Bedroom_Count'
                           ,'FULL BATH': 'Full_Bath_Count'
                           ,'HALF BATH': 'Half_Bath_Count'
                           ,'KITCHEN': 'Kitchen_Count'
                           ,'FIREPLACE': 'Fireplace_Count'
                           ,'REMODEL': 'Remodel'}, inplace=True)

In [ ]:
#Describe Sale Price 
print(df_train.describe())

print(df_train.dtypes)


In [ ]:
#Any NUlls or zeros?
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
zeros = df_train.isin([0]).sum().sort_values(ascending=False)
missing_data = pd.concat([total, percent,zeros], axis=1, keys=['Total', 'Percent','Zero_Count'])
print(missing_data)

In [ ]:
#Lets remove that zero "Year_Built"
df_train = df_train[df_train['Year_Built'] != 0]

In [ ]:
sns.set_theme(style="ticks")
sns.pairplot(df_train,
            y_vars = ["Total_Value"],
            x_vars = ['Tax_Cost', 'Lot_SQFT', 'Year_Built', 'Gross_Area',
       'Living_Area', 'FLOORS ', 'Room_Count', 'Bedroom_Count',
       'Full_Bath_Count', 'Half_Bath_Count', 'Kitchen_Count',
       'Fireplace_Count'])

In [ ]:
'''Learning

Tax_Cost is perfectly linear -- #remove
Lot_SQFT is compact and not as linear as I wanted it to be
Year_Built shows no linear concepts, this was a huge surprise 
to me, but it makes sense, if I have a family of 4 and everyone needs a room
I need 4 rooms, regardless of how large the lot may be.. 
Gross_Area and Living Area are pretty linear! 
Room_Count and Bedroom_Count are linear with a few outliers

#Check correlation of rooms to Gross_Area I assume more rooms == Greater area of houses
'''

In [ ]:
#drop tax cost column
df_train.drop('Tax_Cost', axis=1, inplace=True)



In [ ]:
#Find how correlated heat map
#saleprice correlation matrix
corrmat = df_train.corr()
k = 100 #number of variables for heatmap
cols = corrmat.nlargest(k, 'Total_Value')['Total_Value'].index
cm = np.corrcoef(df_train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()


In [ ]:
'''Learning

WOW - Year_Built is almost meaningless compared to Total_Value
#Check if true 

Living_Area is more correlated than Gross_Area
Rooms are more correlated than bedrooms

Rooms, bedrooms are more valuable than lot square footage

Lot_SQFT is more correlated than bathrooms.. 
I guess we can share a 
bathroom and have more area?
'''

In [ ]:
var = 'Year_Built'
data = pd.concat([df_train['Total_Value'], df_train[var]], axis=1)
f, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x=var, y="Total_Value", data=data);
plt.xticks(rotation=90);

In [ ]:
'''Learning
wow.. yeah that is pretty flat
Year_Built in this dataset is almost meaningless
Houses built in the earlier years 
have similar price dist compared to younger houses
houses built in the middle years are lower
I assume more houses were needed, more were built 
with uniformity and probably the outliers are remodels or
have a larger quanity of rooms/ greater living areas
'''

In [ ]:
#Check for skewness for all columns
numeric_feats = df_train.dtypes[df_train.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = df_train[numeric_feats].apply(lambda x: stats.skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
print(skewness.head(20))

In [ ]:
#Show Distribution of Total Value
#histogram
sns.distplot(df_train['Total_Value'])

#skewness and kurtosis
print("Skewness: %f" % df_train['Total_Value'].skew())
print("Kurtosis: %f" % df_train['Total_Value'].kurt())

In [ ]:
'''Learning
Total_Value is highly '1.3' Positive Skewed
I think this means Houses are being sold for less than the average,
we will need to fix it with logs

Kurtosis > 3 so Total_Value is Leptokurtic
Total_Value does show peakness with tails longer 
on the right side.. check this later.. may
flatten because of outliers

'''

In [ ]:
#Check for Normality
#histogram and normal probability plot
sns.distplot(df_train['Total_Value'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['Total_Value'], plot=plt)


In [ ]:
'''Learning
peaked and positive
does not follow theo lines in prob plot
'''

In [ ]:
#applying log transformation
df_train['Total_Value'] = np.log(df_train['Total_Value'])
#transformed histogram and normal probability plot
sns.distplot(df_train['Total_Value'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['Total_Value'], plot=plt)

In [ ]:
#histogram and normal probability plot
sns.distplot(df_train['Living_Area'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['Living_Area'], plot=plt)

In [ ]:
'''Learning
Living area is very skewed, let's fix with logs  '''

In [ ]:
#data transformation
df_train['Living_Area'] = np.log(df_train['Living_Area'])
#transformed histogram and normal probability plot
sns.distplot(df_train['Living_Area'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['Living_Area'], plot=plt)

In [ ]:
'''Learning

If Living area is this skewed and Gross Area follows
we must test Gross Area'''


In [ ]:
#histogram and normal probability plot
sns.distplot(df_train['Gross_Area'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['Gross_Area'], plot=plt)

In [ ]:
#data transformation
df_train['Gross_Area'] = np.log(df_train['Gross_Area'])
#transformed histogram and normal probability plot
sns.distplot(df_train['Gross_Area'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['Gross_Area'], plot=plt)

In [ ]:
#histogram and normal probability plot
sns.distplot(df_train['Lot_SQFT'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['Lot_SQFT'], plot=plt)


In [ ]:
#data transformation
df_train['Lot_SQFT'] = np.log(df_train['Lot_SQFT'])
#transformed histogram and normal probability plot
sns.distplot(df_train['Lot_SQFT'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['Lot_SQFT'], plot=plt)

In [ ]:
#Let's make dummy variables
#Unique Values in Remodel
print(df_train['Remodel'].unique())

In [ ]:
#Dummy Variables
df_train = pd.get_dummies(df_train)

In [ ]:
'''Now let's do Data Science

My Plan:
I plan to use several methods
Linear Regression (different forms)
Boosted Gradient
BayesianRidge

then simply stack these methods

'''
#Split dataframe into train and test
train, test = train_test_split(df_train, test_size=0.2)

ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.Total_Value.values
y_test = test.Total_Value.values
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['Total_Value'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

train = all_data[:ntrain]
test = all_data[ntrain:]


In [ ]:
#DS libraries
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC, LinearRegression
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb


In [ ]:
#Cross Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [ ]:
'''Before building models I KNOW 
this dataset contains outliers
I know my linear regression models
are sensitive to outliers
so we can make them more robust using
sklearn (Robustscaler()) GBoost(loss = huber)
'''
#Linear Regression
lin_reg = make_pipeline(RobustScaler(), LinearRegression())

#Bayesian Ridge
BayRidge = make_pipeline(RobustScaler(), BayesianRidge(n_iter = 30))

#Lasso Regression
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

#Elastic Net Regression
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

#Kernal Ridge Regression
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

#Gradient Boosting
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213,
                             random_state =7, nthread = -1)

In [ ]:
score = rmsle_cv(lin_reg)
print("\nLinear Regression score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(BayRidge)
print("\nBayesianRidge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
#Simple stacking class and function
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)  



In [ ]:
#We could add many more models but I wanted to have my top 3
averaged_models = AveragingModels(models = (model_xgb, GBoost, KRR))

#Use function to find rsme
score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
'''Learning
So stacking our 3 best performing models
did result in a better score but only slightly

we went from a rough 10.34% average error to
a 9.7% on a 100k house that saves us $500 per house

let's sell 2 a month, 12k a year is a nice bonus 
just from stacking these. With different boosting
params we may get it under 9%
'''

In [ ]:
#Fit models from average
averaged_models.fit(train, y_train)
stacked_train_pred = averaged_models.predict(train)

#Convert to base from logs
stacked_pred = np.expm1(averaged_models.predict(test.values))




In [ ]:
#Dataset setup to have predicted values in df
predicted_df = test.copy(deep=True)
predicted_df['predicted_Total_Value'] = stacked_pred


In [ ]:

#Show Data frame with our predictions
print(predicted_df)

#Prepare dataframe in csv for model submission
predicted_df.to_csv('Submission.csv')

In [ ]:
'''
Let's build a first pass simple linear
regression like what we built in Excel Module 2

Let's use the log values to smooth it out 
'''
#show Linear Regression like the Excel First Pass
regr = LinearRegression()
regr.fit(train, y_train)

#Linear Reg prediction
'''Logs removed use np.expm1'''
#y_predict = np.expm1(regr.predict(test.values))
#y_test = np.expm1(test.Total_Value.values)
y_predict = regr.predict(test.values)

score = rmsle_cv(regr)

#Coefficients
#print("Coefficients: \n", regr.coef_)
cdf = pd.DataFrame(regr.coef_, train.columns, columns = ["Coefficients"])

print(cdf)

#r2 and mean squared values
r_squared = r2_score(y_test,y_predict)
mean_square = mean_squared_error((y_test),(y_predict))
root_mean_square = mean_squared_error((y_test),(y_predict) ,squared = False)

print("\nLinear Regression Mean Error score: {:.4f}\n".format(mean_square))
print("\nLinear Regression Root Mean Error score: {:.4f}\n".format(root_mean_square))
print("\nLinear Regression  R squared score: {:.4f}\n".format(r_squared))


In [ ]:
test_to_prediction_df = pd.DataFrame(np.hstack((y_test)), columns= ["test"])

test_to_prediction_df['prediction'] = np.hstack((y_predict))

test_to_prediction_df['error_value'] = test_to_prediction_df['test']-test_to_prediction_df['prediction']

print("\nSum of errors by subtracting prediction from test values: {:.4f}\n".format(sum(test_to_prediction_df['error_value'])))

In [ ]:
test_to_prediction_df